<a href="https://colab.research.google.com/github/morettitommaso/sql-exercises/blob/main/edb03_Foodstore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caricamento dati nel DBMS

La cella qua sotto serve per per scaricare il database e interagire con il DBMS.

Se il DBMS semtte di funzionare, rieseguite la  cella di codice sottostante.

In [15]:
# Per favore, non modificate il codice di questa cella
#urldb='https://drive.google.com/uc?export=download&id=1S4nPplugS8j7FqvBkV0pz15c1M9U0G1v'
#urldb='https://drive.google.com/uc?export=download&id=1jDXw0RsuDqhrsJZC0PUYCk1bzuJYcbet'
#fn='db03foodstore.db.pez'
#fn='db03foodstore.db.zip'
#https://drive.google.com/file/d/1A4HP78SY3R0uRjevkjhvC6zaxavaVrUa/view?usp=sharing

# Link 1, 2 e 3 attivabili in caso di eccesso di download

# Link 1
#urldb='https://drive.google.com/uc?export=download&id=1A4HP78SY3R0uRjevkjhvC6zaxavaVrUa'
#fn='db03foodstore.db'

#Link 2
#urldb='https://drive.google.com/uc?export=download&id=1S4nPplugS8j7FqvBkV0pz15c1M9U0G1v'
#fn='db03foodstore.db.pez'

#Link 3
urldb='https://www.dropbox.com/scl/fi/nfap938uxptv0pojptufa/db03foodstore.db.pez?rlkey=y5kvhq2nt3w3eod2s18s6wr5z&dl=1'
fn='db03foodstore.db.pez'

pwd='studymat3r1al!'

import sqlite3
import urllib
import urllib.request
import zipfile
import os

import pandas as pd
pd.set_option('display.max_colwidth', None) # -1
pd.set_option('display.max_rows', 100)
from IPython.display import display, HTML

def downloadDBFile(urlFile, localFileName, pwd=''):
  # Dowloading the .zip file into the colab remote computation facility

  print('Downloading db file')
  try:
    urllib.urlretrieve(urlFile, localFileName) # Works in Python2
    print('Python2 detected')
  except AttributeError:
    print('Python 3 detected')
    with urllib.request.urlopen(urlFile) as response: # Works in Python3
      data = response.read()
      fd=open(localFileName,'wb')
      fd.write(data)
    fd.close()
  print('Download Done')
  if localFileName.endswith('.zip'):
    zipFileName = localFileName
    localFileName = zipFileName[:-4]
    print('Unzipping the file content')
    with zipfile.ZipFile(zipFileName, 'r') as zip_ref:
      zip_ref.extractall(localFileName)
      print('File %s extracted' % localFileName)
  elif localFileName.endswith('.pez'):
    zipFileName = localFileName
    localFileName = zipFileName[:-4]
    print('Unzipping the password protected file content')
    with zipfile.ZipFile(zipFileName, 'r') as zip_ref:
      #zip_ref.extractall(localFileName, pwd=bytes(pwd,'utf-8')) # it messes up the paths with password encrypted files
      zip_ref.extract(localFileName, pwd=bytes(pwd,'utf-8')) # localFileName: the name of the file in the .zip archive
      print('File %s extracted' % localFileName)
  elif localFileName.endswith('.db'):
    pass

  localFileName = os.path.abspath(localFileName)
  print('File saved in '+localFileName)
  return localFileName
  print('Unzipping the password protected file content')


dbFileFullPath = downloadDBFile(urldb, fn, pwd=pwd)

def rq(query, threshold=200): # eq is for Run Query
    conn = sqlite3.connect(dbFileFullPath)
    if query.replace(' ','').replace('\n','') == '':
      return
    try:
        res = pd.read_sql(query,conn)
        conn.close()
        nrecords=res.shape[0]
        if nrecords<=threshold:
          display(res)
          print("%d records returned" % (nrecords) )
          #HTML("<b> %d records returned </b>" % (nrecords) )
          #return res

        else:
          display(res.iloc[:threshold,:])
          print("Too many records in the result, returned only the first %d" % (threshold))
          #return res.iloc[:threshold,:]

    except (pd.io.sql.DatabaseError, Exception)  as dbe: # (sqlite3.DatabaseError , sqlite3.OperationalError)
        print('Error')
        print(type(dbe))
        print(dbe)

Python 3 detected
Download Done
Unzipping the password protected file content
File db03foodstore.db extracted
File saved in /content/db03foodstore.db


# EDB03 Foodstore

In questo esercizio dovrete interagire con un database di nome *foodstore* composto da diverse tabelle.

## Terminologia

Il database foodstore che memorizza i dati delle vendite di un supermercato. Le tabelle e i dati sono in lingua inglese, trovate qui di seguito la traduzione in italiano di alcuni dei termini usati nei nomi delle tabelle e nei nomi degli attributi del database


* Bill (Scontrino)
* Customer (Cliente)
* Education (Titolo di studio)
* Item in bill (Articolo nello scontrino)
* Member_card (Tessera di fidelizzazione)
* Supplier (Fornitore)


## Database Schema

<img src="https://drive.google.com/uc?export=download&id=1FFdUQW8ohAhAyU2EdUpzek3p3AyB83L-" alt="Database Schema" width="800">

Potete aprire l'immagine in un'altra pagina seguendo [questo link](https://drive.google.com/file/d/1FFdUQW8ohAhAyU2EdUpzek3p3AyB83L-/view?usp=sharing).

In [22]:
rq("""
select * from member_card;
""")

,card_type_id,description
0,1,Normal
1,2,Bronze
2,3,Silver
3,4,Gold


4 records returned


# Esercizi

Costruite delle query SQL per rispondere alle seguenti domande.

Negli esercizi da 1 a 6 è possibile contare gli elementi in due modi:

   * guardando quanti sono i record restituiti nell’output sintetico,

   * scrivendo una query che utilizza le funzioni di aggregazione (solo se vi sono state spiegate).

Dall’esercizio 7 in avanti è necessario conoscere le funzioni di aggregazione. Se avete svolto gli esercizi da 1 a 6 senza usare le funzioni di aggregazione, una volta studiate, provate a svolgere nuovamente gli esercizi applicandole dove possibile. In alcuni casi, l'output ottenuto con le funzioni di aggregazione è diverso e non risponde pienamente alle richieste delle domande da 1 a 6 (es., nell'output mancheranno i dettagli dei singoli record di partenza), pertanto consideratelo come un esercizio extra.

**Riepilogo sintassi query SQL**

SELECT Attributo1, Attributo2, ..., *

FROM Tabella1, Tabella2, …, TabellaN

WHERE CondizioniSemplici

ORDER BY ListaAttributiDiOrdinamento

;

## (EDB03) 1

Quanti sono i clienti con nome “Charles”?, quanti con nome “Patricia”? quanti con nome “Sharon”?

Le informazioni sui clienti sono memorizzate nella tabella Customer.

Gli attributi lname (last name) e fname (first name) rappresentano rispettivamente il cognome e il nome dei clienti.

Nota: Charles 5, Patricia 4, Sharon 3.


In [31]:
rq("""
select fname, count(*) as total
from customer
where fname in ("Charles" ,"Patricia" ,"Sharon")
group by fname;
""")
# Potete aggiungere altre caselle di codice qua sotto per svolgere altre query

,fname,total
0,Charles,5
1,Patricia,4
2,Sharon,3


3 records returned


## (EDB03) 2

Quante sono le persone con tessera “Gold”? Quante con tessera “Silver”, quante con tessera “Bronze”, quante con tessera “Normal”?

Nota:
Normal 98, Bronze 159, Silver 21, Gold assenti.


In [35]:
rq("""
select * from member_card
""")
# Potete aggiungere altre caselle di codice qua sotto per svolgere altre query

,card_type_id,description
0,1,Normal
1,2,Bronze
2,3,Silver
3,4,Gold


4 records returned


In [38]:
rq("""
select customer.card_type_id, count(*) as total
from customer, member_card
where customer.card_type_id = member_card.card_type_id
group by customer.card_type_id
""")

,card_type_id,total
0,1,98
1,2,159
2,3,21


3 records returned


## (EDB03) 3

Con un'unica query, visualizzate i dati degli scontrini (nello specifico gli attributi *bill_id*, *date* e *total* della tabella bill) emessi sul cliente “Clyde Dixon” (Dixon è il cognome) e sul cliente “Bonnie Emerson” (Emerson è il cognome). La stessa query dovrà visualizzare, per ogni scontrino, anche il nome, il cognome e il customer_id del cliente.

Prima di eseguire la query, (se vi sono state spiegate le funzioni di aggregazione) domandatevi: è necessario utilizzare le funzioni di aggregazione per rispondere alla domanda precedente?

Nota, nel risultato devono essere presenti 5 record per il cliente Dixon e 9 record per il cliente Emerson (**14 record in totale**).

Se avete visto le funzioni di aggregazione, come esercizio extra, potete scrivere una query che conti numero di scontrini emessi per ognuno dei due clienti. In quest'ultima query (quella che utilizza le funzioni di aggregazione), sono attesi 2 record nel risultato.


In [40]:
rq("""
SELECT bill.bill_id, bill.date, bill.total, customer.lname, customer.fname
FROM bill, customer
WHERE (customer.customer_id = bill.customer_id) AND
((fname="Clyde" AND lname="Dixon") OR (fname="Bonnie" AND lname="Emerson"));
""")
# Potete aggiungere altre caselle di codice qua sotto per svolgere altre query

,bill_id,date,total,lname,fname
0,256,1997-02-12,42.35,Dixon,Clyde
1,341,1997-02-19,103.78,Dixon,Clyde
2,502,1997-03-22,65.34,Emerson,Bonnie
3,505,1997-03-22,105.37,Dixon,Clyde
4,813,1997-05-03,10.08,Emerson,Bonnie
5,853,1997-05-05,133.84,Dixon,Clyde
6,888,1997-05-09,73.60,Emerson,Bonnie
7,1138,1997-06-22,23.31,Emerson,Bonnie
8,1270,1997-07-04,132.68,Emerson,Bonnie
9,1363,1997-07-06,19.62,Dixon,Clyde


14 records returned


In [42]:
rq("""
SELECT lname, fname, count(*) as "numero scontrini"
FROM bill, customer
WHERE (customer.customer_id = bill.customer_id) AND
((fname="Clyde" AND lname="Dixon") OR (fname="Bonnie" AND lname="Emerson"))
GROUP BY lname;
""")

,lname,fname,numero scontrini
0,Dixon,Clyde,5
1,Emerson,Bonnie,9


2 records returned


## (EDB03) 4

Visualizzate i dati degli scontrini (tabella bill, ... vedi esercizio precedente) che riportano un totale superiore a 307 dollari. Nella tabella risultato ogni scontrino dovrà essere rappresentato da un record distinto (non servono funzioni di aggregazione), ogni record dovrà visualizzare i dati sia dello scontrinto (es., total, date, bill_id) sia del cliente (nome, cognome, customer_id).

Guardando i risultati della query (non serve scrivere un'atra query), quante volte appare il nome di “Bradley Doman” (Doman è il cognome)?

Nota: 18 scontrini con importo superiore a 307 dollari, di cui 2 emessi a Bradley Doman


In [44]:
rq("""
SELECT bill.bill_id, bill.date, bill.total, customer.lname, customer.fname, customer.customer_id
FROM bill, customer
WHERE (customer.customer_id = bill.customer_id)
AND (bill.total > 307);
""")

,bill_id,date,total,lname,fname,customer_id
0,61,1997-01-13,340.80,Chartier,Jim,6760
1,83,1997-01-16,472.42,Bandy,Deirdre Susan,1328
2,413,1997-02-25,329.32,Doman,Bradley,9715
3,441,1997-03-18,364.42,Styers,Jeremy,7556
4,580,1997-03-28,371.10,Voerge,Bernard,6799
5,678,1997-04-07,370.06,Stanley,Charles,2942
6,952,1997-05-14,307.78,Scott,James,9271
7,1083,1997-06-16,321.35,Staples,Tony,9837
8,1193,1997-06-27,307.10,Surber,Charles,5877
9,1348,1997-07-05,308.73,Hover,Dennis,9327


18 records returned


## (EDB03) 5

Visualizzate il cognome ed il nome di tutti i clienti che hanno acquistato il prodotto "Great Muffins".

Nota: 9 persone diverse nel risultato.


In [46]:
rq("""
SELECT fname, lname
FROM product, item_in_bill, bill, customer
WHERE (product.product_id = item_in_bill.product_id)
AND (item_in_bill.bill_id = bill.bill_id)
AND (bill.customer_id = customer.customer_id)
AND product.product_name="Great Muffins";
""")

,fname,lname
0,Agnes,Pfanz
1,Joseph,Cantoni
2,Linda,Basile
3,Jack,Parente
4,Ethel,Reiners
5,Cheryl,Vigil
6,Michael,Swartwood
7,Christian,Snowdall
8,Lori,Merrill


9 records returned


## (EDB03) 6

Visualizzate il cognome e il nome di tutti i clienti che hanno acquistato prodotti forniti dall’azienda Bravo. Nel risultato, visualizzate anche i nomi dei prodotti acquistati e la data in cui è avvenuto l’acquisto. I prodotti acquistati più volte si devono vedere distintamente (non sono richieste operazioni di aggregazione).

Nota: 152 record nel risultato.


In [47]:
rq("""

SELECT fname, lname, product_name, date

FROM product, item_in_bill, bill, customer, supplier

WHERE (supplier.supplier_id = product.supplier_id)

AND (product.product_id = item_in_bill.product_id)

AND (item_in_bill.bill_id = bill.bill_id)

AND (bill.customer_id = customer.customer_id)

AND supplier.name="Bravo"

;

""")

,fname,lname,product_name,date
0,Joseph,Cantoni,Bravo Canned String Beans,1997-01-13
1,Eric,Coleman,Bravo Fancy Canned Sardines,1997-01-13
2,Thomas,Duran,Bravo Rice Soup,1997-01-13
3,Jim,Chartier,Bravo Creamed Corn,1997-01-13
4,Helen,Laing,Bravo Fancy Canned Anchovies,1997-01-26
...,...,...,...,...
147,Carol,York,Bravo Chicken Soup,1997-12-25
148,Bertha,Wilcox,Bravo Fancy Canned Clams,1997-12-25
149,Beverly,Christensen,Bravo Canned Yams,1997-12-25
150,Catherine,Gilbert,Bravo Creamed Corn,1997-12-25


152 records returned


## (EDB03) 7

Per ogni prodotto con product_id inferiore a 20, visualizzate la quantità totali di prodotto venduta. Per ogni prodotto deve apparire un solo record nel risultato, i record devono essere ordinati in ordine decrescente rispetto alla quantità venduta.

Nota: il prodotto più venduto è "Washington Cola" con 36 unità.


In [51]:
rq("""

SELECT product_name, SUM(item_in_bill.quantity) as QT_TOTALE

FROM product, item_in_bill, bill

WHERE (product.product_id = item_in_bill.product_id)

AND (item_in_bill.bill_id = bill.bill_id)

AND product.product_id < 20

GROUP BY product.product_id
ORDER BY QT_TOTALE DESC

;

""")

,product_name,QT_TOTALE
0,Washington Cola,36
1,Washington Cream Soda,32
2,Jeffers Grits,27
3,Washington Diet Cola,27
4,Blue Label Creamed Corn,26
5,Jeffers Oatmeal,25
6,Jeffers Corn Puffs,24
7,Washington Apple Juice,22
8,Washington Strawberry Drink,22
9,Washington Apple Drink,21


19 records returned


## (EDB03) 8

Individuare la classe di prodotti che vende di più in termini di quantità. Le informazioni sulla classe di prodotti sono contenute nella tabella product_class, (chiave: product_class_id, nome della classe di prodotti nell’attributo product_subcategory).  Ordinate i risultati in ordine decrescente di quantità vendute.

Nota: la classe di prodotti che vende più quantità è: Product_class_id 61, product_subcategory “Fresh Vegebales”, Total QT 2668.

Nota 2: Se il dbms restituisce un errore "GROUP BY clause contains nonaggregated column 'foodstore.pc.product_subcategory' ", inserite anche product_subcategory nel group by.

In [54]:
rq("""

SELECT product_category, SUM(item_in_bill.quantity) as QT_TOTALE

FROM product_class, product, item_in_bill

WHERE (product_class.product_class_id = product.product_class_id)

AND (product.product_id = item_in_bill.product_id)

GROUP BY product_class.product_class_id
ORDER BY QT_TOTALE DESC

;

""")

,product_category,QT_TOTALE
0,Vegetables,2668
1,Fruit,1527
2,Dairy,1123
3,Canned Soup,1094
4,Snack Foods,1079
...,...,...
97,Seafood,88
98,Breakfast Foods,81
99,Candles,80
100,Canned Oysters,66


102 records returned


## (EDB03) 9

Ora calcoleremo il ricavo generato da un prodotto. Se vi sentite molto ferrati sulle funzioni di aggregazione potete saltare direttamente all'ultimo sottopunto, altrimenti svolgete tutti i sottopunti seguenti.

#### (EDB03) 9.a

Visualizzate tutti i record della tabella *item_in_bill* relativi a vendite del prodotto con product_id=205. Visualizzate solamente gli attributi product_id, quantity e price. Price rappresenta il prezzo unitario di vendita.

Nota: 2 record nel risultato

In [55]:
rq("""

SELECT product_id, quantity, price
FROM item_in_bill
WHERE product_id=205;

""")

,product_id,quantity,price
0,205,4,10.84
1,205,2,5.42


2 records returned


#### (EDB03) 9.b

Come la query precedente, tuttavia aggiungete agli attributi visualizzati nel risultato **il prodotto** (nel senso di moltiplicazione) dell'attributo *quantity* per l'attributo *price*.


In [56]:
rq("""

SELECT product_id, (quantity * price)
FROM item_in_bill
WHERE product_id=205;

""")

,product_id,(quantity * price)
0,205,43.36
1,205,10.84


2 records returned


#### (EDB03) 9.c

Sempre a partire dalla tabella *item_in_bill*, per il prodotto con product_id=205 visualizzate il product_id, il totale delle quantità vendute e il totale dei soldi incassati dalla vendita prodotto (cioè il ricavo totale generato dal prodotto).

Nota. Nel risultato, product_id 205, totale qt vendute 6, ricavo totale 54,20

In [59]:
rq("""

SELECT product_id, SUM(quantity) as TOTAL_QT, SUM(price*quantity) AS TOTAL_EARN
FROM item_in_bill
WHERE product_id=205;
""")

,product_id,TOTAL_QT,TOTAL_EARN
0,205,6,54.2


1 records returned


#### (EDB03) 9.d

Nella query precedente, avrebbe senso calcolare sum(price)? Perché?

Scrivete la vostra risposta in questa cella (doppio click per entrare in modalità editing)

#### (EDB03) 9.e

Riprendete la query del punto 9.c e aggiungete un ulteriore attributo: sum(quantity) * price. Provate ad eseguire. L'ultimo attributo aggiunto è un modo corretto di calcolare il ricavo totale generato dal prodotto? Perché?

In [60]:
rq("""

SELECT product_id,
      SUM(quantity) as TOTAL_QT,
      SUM(price*quantity) AS TOTAL_EARN,
      sum(quantity) * price

FROM item_in_bill
WHERE product_id=205;
""")

,product_id,TOTAL_QT,TOTAL_EARN,sum(quantity) * price
0,205,6,54.2,65.04


1 records returned


 #### (EDB03) 9.f

Visualizzate tutti i record della tabella *item_in_bill* relativi a vendite dei prodotti con id 205 e 188. Visualizzate solamente gli attributi product_id, quantity, price e infine quantity moltiplicato per price.

Nota: 3 record nel risultato

In [62]:
rq("""

SELECT product_id,
      price,
      quantity,
      quantity * price

FROM item_in_bill
WHERE product_id=205
      OR product_id=188;
""")

,product_id,price,quantity,quantity * price
0,188,10.32,4,41.28
1,205,10.84,4,43.36
2,205,5.42,2,10.84


3 records returned


#### (EDB03) 9.g

A partire dalla tabella *item_in_bill*, visualizzate il totale delle quantità vendute e il totale dei soldi incassati dalla vendita dei prodotti con id 205 e 188. Nel risultato deve essere presente un solo record.

Nota. Qt vendute 10, Ricavo 95.48

In [63]:
rq("""

SELECT product_id,
      SUM(quantity) as TOTAL_QT,
      SUM(price*quantity) AS TOTAL_EARN

FROM item_in_bill
WHERE product_id=205
      OR product_id=188;

""")

,product_id,TOTAL_QT,TOTAL_EARN
0,188,10,95.48


1 records returned


#### (EDB03) 9.h

A partire dalla tabella *item_in_bill*, per i prodotti con id 205 e 188 visualizzate il totale delle quantità vendute e il totale dei soldi incassati dalla vendita, *distinti per prodotto* (cioè ogni prodotto deve avere i suoi totali). Nel risultato devono essere presenti due record.

Nota. Fate riferimento ai risultati delle query precedenti per confrontare i risultati

In [64]:
rq("""

SELECT product_id,
      SUM(quantity) as TOTAL_QT,
      SUM(price*quantity) AS TOTAL_EARN

FROM item_in_bill
WHERE product_id=205
      OR product_id=188

GROUP BY product_id;

""")

,product_id,TOTAL_QT,TOTAL_EARN
0,188,4,41.28
1,205,6,54.20


2 records returned


#### (EDB03) 9.i

Calcolate per ogni prodotto, il ricavo totale. Nella query risultato deve apparire un solo record per ogni prodotto, con indicato: il product_id e il ricavo totale generato dal prodotto. Ordinate i risultati in ordine decrescente rispetto al ricavo totale.

Nota: product_id 1415 con ricavo totale 619.40 è il prodotto che fa incassare più soldi

In [67]:
rq("""

SELECT product_id,
      SUM(price*quantity) AS TOTAL_EARN

FROM item_in_bill
GROUP BY product_id
ORDER BY TOTAL_EARN DESC;

""")

,product_id,TOTAL_EARN
0,1415,619.40
1,1428,590.58
2,302,569.25
3,975,560.24
4,1448,547.66
...,...,...
195,162,276.45
196,450,276.34
197,1366,276.00
198,1011,275.79


Too many records in the result, returned only the first 200


#### (EDB03) 9.j

Come la query precedente, tuttavia nel risultato visualizzate anche il nome del prodotto.

In [70]:
rq("""

SELECT product.product_id,
      SUM(price*quantity) AS TOTAL_EARN,
      product.product_name

FROM item_in_bill,
      product

WHERE (item_in_bill.product_id == product.product_id)
GROUP BY item_in_bill.product_id
ORDER BY TOTAL_EARN DESC;

""")

,product_id,TOTAL_EARN,product_name
0,1415,619.40,Gauss Monthly Sports Magazine
1,1428,590.58,Hermanos Green Pepper
2,302,569.25,Super Oregano
3,975,560.24,Just Right Rice Soup
4,1448,547.66,Hermanos Lemons
...,...,...,...
195,162,276.45,Denny Large Sponge
196,450,276.34,Red Wing Paper Plates
197,1366,276.00,Hilltop Deodorant
198,1011,275.79,American Sliced Turkey


Too many records in the result, returned only the first 200


## (EDB03) 10

Individuare la classe di prodotti che vende di più in termini di ricavo. Le informazioni sulla classe di prodotti sono contenute nella tabella *product_class*, (chiave: product_class_id, nome della classe di prodotti nell’attributo product_subcategory).  Ordinate i risultati in ordine decrescente rispetto al ricavo.

Nota: la classe di prodotti che vende più è: product_class_id 61, product_subcategory “Fresh Vegebales”, ricavo totale 19364.56.

Nota 2: Se il dbms restituisce un errore "GROUP BY clause and contains nonaggregated column 'foodstore.pc.product_subcategory' ", inserite anche product_subcategory nel group by.

In [72]:
rq("""

SELECT product_category, SUM(item_in_bill.quantity * item_in_bill.price) as TOTAL_EARN

FROM product_class, product, item_in_bill

WHERE (product_class.product_class_id = product.product_class_id)

AND (product.product_id = item_in_bill.product_id)

GROUP BY product_class.product_class_id
ORDER BY TOTAL_EARN DESC

;

""")

,product_category,TOTAL_EARN
0,Vegetables,19364.56
1,Fruit,11739.51
2,Dairy,8902.55
3,Snack Foods,8177.46
4,Canned Soup,7442.90
...,...,...
97,Bathroom Products,525.19
98,Breakfast Foods,520.06
99,Seafood,494.47
100,Canned Oysters,438.23


102 records returned
